## MiniCaml con Insiemi

Vogliamo estendere l'interprete visto a lezione, implementando un costrutto che permetta di formare **insiemi di elementi dello stesso tipo**. Un insieme è una sequenza di elementi **senza duplicati**, e può contenere unicamente **interi, booleani e stringhe**. Esempi di espressioni in questo linguaggio esteso potrebbero essere:
```
{int: 1, 2, 3, 4};;

{int:};;

{int: 4};;

let A = {int: 1, 2, 3, 4} in 
    insert(6, A);;

let A = {string: "hello", "world", "from", "MiniCaml" } in
    if ("hi" in A) then 0 else 1

let A = {string: "hello", "world", "from", "MiniCaml" } in
let B = {string: "hi" "world"} in
    intersection A B
```

### Es 1: Albero della sintassi astratta

Estendere l'AST di MiniCaml con un **nuovo tipo** e le seguenti **nuove espressioni**:
1. Espressioni per **costruire insiemi** (composti unicamente con interi, booleani e stringhe):
    - una per l'insieme vuoto (di un certo tipo) 
    - una per l'insieme singoletto contenente un dato elemento (di un certo tipo) 
    - e una per l'insieme contenente una data sequenza di elementi distinti (tutti di un certo tipo)
    
    __NOTA:__ Ogni insieme, anche se vuoto, mantiene sempre l'informazione sul tipo di elementi che può contenere. Quindi le espressioni `{int:}` e `{string:}` dovranno essere di tipi diversi...


2. Un **operatore unario** su insiemi:
    - `isEmpty`, che restituisca true solo se l'insieme è vuoto


3. **Operatori fra insiemi ed elementi**:
    - `Insert`, tale che `insert x A` resituisca $A \cup \{x\}$
    - `Remove`, tale che `remove x A` restituisca $A \setminus \{x\}$
    - `Member`, tale che `member x A` restituisca `true` sse $x \in A$


4. **Operatori binari** su insiemi:
    - `Union`, tale che `union A B` restituisca $A \cup B$
    - `Intersection`, tale che `intersection A B` restituisca $A \cap B$
    - `Subset`, tale che `subset A B` restituisca `true` sse $A \subset B$

#### Soluzione:

Nel definire un nuovo tipo per gli insiemi, non possiamo aggiungere  a `tname` un semplice costruttore `TSet`, poichè un insieme di interi non è un insieme di stringhe e viceversa. Dobbiamo quindi creare un tipo che sia **parametrico rispetto a un altro tipo**. `TSet` è infatti un *type constructor*, ovvero una sorta di "funzione" che, preso in input un tipo, restituisce un nuovo tipo. Lo stesso meccanismo è presente in Ocaml con *type constructors* come `list` e `option`. 

Dobbiamo quindi estendere `tname` con un costruttore che contenga un altro tname, ad esempio `TSet of tname`. Così facendo, però, potremmo rappresentare anche tipi che non vogliamo all'interno di MiniCaml, come `TSet(TClosure)`, ovvero il tipo "insieme di funzioni" (vogliamo solo insiemi di interi, booleani e stringhe). Una possibile soluzione è lasciare `TSet of tname`, e controllare più avanti, nella fase di typechecking e valutazione, che siano formati solo insiemi del giusto tipo. La soluzione che vedremo, invece, è estendere `tname` con `TSet of tsub`, dove `tsub` è un nuovo variant type (*in Ocaml*) che ha un sottoinsieme dei costruttori presenti in `tname`.

Questa soluzione ha un vantaggio rispetto all'altra: fa uso del typechecking di Ocaml, per il quale sarà *sintatticamente* impossibile costruire un tipo `TSet(TClosure)`, senza dover riempire di if la parte semantica, e rischiare di dimenticarsene qualcuno. Uno svantaggio è che ora ci sono due diverse concezioni di `TInt`, quella dentro `tname` e quella dentro `tsub`, quando invece per il l'utente di MiniCaml esiste un solo tipo intero. Ci occuperemo di questa particolarità più avanti.

Definito il nuovo tipo, è immediato definire i nuovi nodi dell'AST, ricordandosi che ogni insieme, anche vuoto, deve contenere sempre l'informazione sul tipo degli elementi contenuti.

In [1]:
(* Identificatori *)
type ide = string;;

(* I tipi *)
type tsub =  TInt | TBool | TString;;
type tname =  TInt | TBool | TString | TClosure | TRecClosure | TUnBound | TSet of tsub;;

(* Abstract Expressions = espressioni nella sintassi astratta, 
   compongono l'Albero di Sintassi Astratta *)
type exp = 
    | EInt of int
    | CstTrue 
    | CstFalse
    | EString of string
    | Den of ide
    (* Operatori binari da interi a interi*)
    | Sum of exp * exp
    | Diff of exp * exp
    | Prod of exp * exp
    | Div of exp * exp
    (* Operatori da interi a booleani *)
    | IsZero of exp
    | Eq of exp * exp
    | LessThan of exp*exp
    | GreaterThan of exp*exp
    (* Operatori su booleani *)
    | And of exp*exp
    | Or of exp*exp
    | Not of exp
    (* Controllo del flusso, funzioni *)
    | IfThenElse of exp * exp * exp
    | Let of ide * exp * exp
    | Letrec of ide * ide  * exp * exp
    | Fun of ide * exp
    | Apply of exp * exp
    (* costruttori di insiemi *)
    (**)
    | Empty of tsub (*formalizza il caso {int: }*)
    | Singleton of tsub*exp (*formalizza il caso {int: 3}*)
    | Of of tsub*exp list (*formalizza il caso {int: 3, 1, 4}*)
    (*operatori*)
    |IsEmpty of exp
    |Insert of exp*exp
    |Remove of exp*exp
    |Member of exp*exp
    |Union of exp*exp
    |Intersection of exp*exp 
    |IsSubset of exp*exp 

type ide = string


type tsub = TInt | TBool | TString


type tname =
    TInt
  | TBool
  | TString
  | TClosure
  | TRecClosure
  | TUnBound
  | TSet of tsub


type exp =
    EInt of int
  | CstTrue
  | CstFalse
  | EString of string
  | Den of ide
  | Sum of exp * exp
  | Diff of exp * exp
  | Prod of exp * exp
  | Div of exp * exp
  | IsZero of exp
  | Eq of exp * exp
  | LessThan of exp * exp
  | GreaterThan of exp * exp
  | And of exp * exp
  | Or of exp * exp
  | Not of exp
  | IfThenElse of exp * exp * exp
  | Let of ide * exp * exp
  | Letrec of ide * ide * exp * exp
  | Fun of ide * exp
  | Apply of exp * exp
  | Empty of tsub
  | Singleton of tsub * exp
  | Of of tsub * exp list
  | IsEmpty of exp
  | Insert of exp * exp
  | Remove of exp * exp
  | Member of exp * exp
  | Union of exp * exp
  | Intersection of exp * exp
  | IsSubset of exp * exp


### Es 2: Tipi esprimibili

Estendere i tipi esprimibili e la funzione `getType` per comprendere anche i valori insieme.

#### Soluzione:

Nell'estendere `evT`, va fatta una importante scelta implementativa: come rappresentare un valore semantico "insieme". Gli interi di Minicaml possono essere implementati come interi in Ocaml, e similmente le stringhe e così via. Non essendoci una struttura dati nativa degli insiemi in Ocaml, dobbiamo scegliere noi che struttura dati utilizzare. La scelta più semplice (ma non computazionalmente ottimale) è una lista non ordinata, ma si potrebbero usare anche liste ordinate, hashmap, BST e simili. Passare da una implementazione all'altra dovrebbe avere effetto solo su `evT` e sulle primitive che manipolano oggetti di tipo `evT`, il resto del codice sulla sintassi o sulla valutazione rimarrebbe identico.

In [2]:
(* Ambiente polimorfo *)
type 't env = ide -> 't

(* Evaluation types = tipi  esprimibili *)
type evT = 
    | Int of int 
    | Bool of bool 
    | String of string 
    | Closure of ide * exp * evT env 
    | RecClosure of ide * ide * exp * evT env
    | Set of tsub*(evT list)
    | UnBound

(* Ambiente vuoto *)
let emptyenv = function x -> UnBound

(* Binding fra una stringa x e un valore primitivo evT *)
let bind (s: evT env) (x: ide) (v: evT) = 
    function (i: ide) -> if (i = x) then v else (s i)

(* Funzione da evT a tname che associa a ogni valore il suo descrittore di tipo  *)
let getType (x: evT) : tname =
    match x with
    | Int(_) -> TInt
    | Bool(_) -> TBool
    | String(_) -> TString
    | Closure(i,e,en) -> TClosure
    | RecClosure(i,j,e,en) -> TRecClosure
    | Set(t, _) -> TSet(t)
    | UnBound -> TUnBound

type 't env = ide -> 't


type evT =
    Int of int
  | Bool of bool
  | String of string
  | Closure of ide * exp * evT env
  | RecClosure of ide * ide * exp * evT env
  | Set of tsub * evT list
  | UnBound


val emptyenv : 'a -> evT = <fun>


val bind : evT env -> ide -> evT -> ide -> evT = <fun>


val getType : evT -> tname = <fun>


### Es 3: Typechecking

Estendere la funzione di typechecking in maniera tale che `typecheck(TSet, v)` restituisca `true` se e solo se `v` è un valore di tipo `TSet`, cioè è un insieme di valori, e tutti gli elementi contenuti in `v` sono del tipo adeguato.

NOTA: Può essere utile scrivere una funzione ```upcast: tsub -> tname```.

#### Soluzione:

Se vogliamo assicurarci che un valore `Set(t', l) : evT` sia di tipo `TSet(t')`, dobbiamo assicurarci che
 1. `t' = t`, perché un set di interi non può in nessun caso essere di tipo "set di stringhe"
 2. tutti gli elementi in `l` siano di tipo `t`. Qui compare però una peculiartià: a ogni elemento `v` dentro `l` posso associare il suo tipo con la funzione `gettype: evT -> tname`, ma dentro il tipo `TSet(t)`, `t` non è un `tname`, ma un `tsub`! Cioè mi devo assicurare che, per ogni `v` in `l`, `gettype v = t`, ma `getype v` e `t` sono incomparabili, per Ocaml. Dobbiamo quindi definire una funzione `upcast: tsub -> tname`, che effettui l'ovvia conversione, così da poter verificare che, per ogni `v` in `l`, `getype v = upcast t`.
 
     Upcast non è una funzione che ha alcuna valenza semantica, perchè in MiniCaml c'è una sola nozione di "tipo intero", è solo una peculiarità della nostra implementazione.
 

In [4]:
(* Errori a runtime *)
exception RuntimeError of string

let (upcast: tsub -> tname) = function
      TInt -> TInt
    | TBool -> TBool
    | TString -> TString
    

(* Type-checking *)
let typecheck ((x, y) : (tname*evT)) = 
    match x with
    | TInt -> (match y with 
               | Int(u) -> true
               | _ -> false
               )
    | TBool -> (match y with 
                | Bool(u) -> true
                | _ -> false
                )
    | TString -> (match y with
                 | String(u) -> true
                 | _ -> false
                 )
    | TClosure -> (match y with
                   | Closure(i,e,n) -> true
                   | _ -> false
                   )
    | TRecClosure -> (match y with
                     | RecClosure(i,j,e,n) -> true
                     | _ -> false
                     )
    | TUnBound -> (match y with
                 | UnBound -> true
                 | _ -> false
                 )
    |TSet(t) -> (match y with
                | Set(t', list) -> t = t' && List.for_all (fun x -> getType x = upcast t) list
                | _ -> false)
                

exception RuntimeError of string


val upcast : tsub -> tname = <fun>


val typecheck : tname * evT -> bool = <fun>


### Utilities

Seguono alcune utilities sulle liste, già implementate.

In [5]:
(* UTILITIES per le primitive del linguaggio (NON sono primitive!) *)

(* Controlla se una lista contiene o meno l'elemento dato *)
let rec list_contains l x = match l with
    | [] -> false
    | p::q -> if x = p then true else list_contains q x

(* Controlla che una lista non abbia elementi ripetuti *)
let rec checkNotEquals l = match l with
    | [] -> true
    | p::q -> if (not(list_contains q p)) then checkNotEquals q else false 

(* Requires: l1 ed l2 non hanno elementi ripetuti al loro interno
Controlla se tutti gli elementi di l1 sono contenuti in l2 o meno *)
let rec is_contained l1 l2 = match l1 with
    | [] -> true
    | p::q -> if (list_contains l2 p) then (is_contained q l2) else false

(* Rimuove l'elemento x dalla lista l se presente, dà errore altrimenti *)
let rec list_remove l x = match l with
    | [] -> raise ( RuntimeError "")
    | p::q -> if (p = x) then q else p::(list_remove q x)

val list_contains : 'a list -> 'a -> bool = <fun>


val checkNotEquals : 'a list -> bool = <fun>


val is_contained : 'a list -> 'a list -> bool = <fun>


val list_remove : 'a list -> 'a -> 'a list = <fun>


### Es 4: Primitive

Implementare le primitive per gli insiemi, quali
- Creazione
- Inserzione, rimozione e appartenenza
- ```is_subset``` e ```is_empty```
- Unione intersezione

#### Soluzione:

Le primitive sono le operazioni che la macchina virtuale è capace di compiere, e il processo di valutazione (la fnzione `eval`) associa a ogni espressione sintattica un valore, componendo insieme più primitive. Lo stesso approccio avviene nel processore, che carica dalla memoria un'istruzione, la decodifica, e sceglie quali primtive (implimentate dalla ALU) eseguire per produrre un risultato.


Dobbiamo sostanzialmente implementare una primitiva per ogni costrutto semantico definito dentro exp, avvalendoci delel funzioni di utilities. Tutte le primitive accettano un certo numero di parametri (i.e. operandi), ne fanno il typechecking, e poi resituiscono il risultato.

In [9]:
(* PRIMITIVE del linguaggio *)

(* Controlla se un numero è zero *)
let is_zero(x) = match (typecheck(TInt,x),x) with
    | (true, Int(v)) -> Bool(v = 0)
    | (_, _) -> raise ( RuntimeError "Wrong type")

(* Uguaglianza fra interi *)
let int_eq(x,y) =   
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v = w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Somma fra interi *)	   
let int_plus(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v + w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Differenza fra interi *)
let int_sub(x, y) = 
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v - w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Prodotto fra interi *)
let int_times(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Int(v * w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Divisione fra interi *)
let int_div(x, y) =
    match(typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> 
                    if w<>0 then Int(v / w)
                            else raise (RuntimeError "Division by zero")
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni di confronto *)
let less_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v < w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let greater_than(x, y) = 
    match (typecheck(TInt,x), typecheck(TInt,y), x, y) with
    | (true, true, Int(v), Int(w)) -> Bool(v > w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

(* Operazioni logiche *)
let bool_and(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v && w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_or(x,y) = 
    match (typecheck(TBool,x), typecheck(TBool,y), x, y) with
    | (true, true, Bool(v), Bool(w)) -> Bool(v || w)
    | (_,_,_,_) -> raise ( RuntimeError "Wrong type")

let bool_not(x) = 
    match (typecheck(TBool,x), x) with
    | (true, Bool(v)) -> Bool(not(v))
    | (_,_) -> raise ( RuntimeError "Wrong type")
    
(* Crea un nuovo insieme vuoto *)
let new_empty (t : tsub) = Set(t, [])

(* Crea un nuovo insieme con un solo elemento *)
let new_singleton ((t, e) : (tsub*evT)) =
    (*Nel typechecking, dobbiamo assicurarci che e sia di tipo t, 
      ma e e t sono incomparabili, per questo motivo vanno confrontati e e upcast t*)
    if typecheck(upcast t, e) then Set(t, [e])
    else raise ( RuntimeError "Wrong type")

(* Crea un nuovo insieme partendo da una lista di elementi. *)
let new_of ((t , l) : (tsub*(evT list))) = 
    if checkNotEquals l 
        then 
            (*l'operazione corretta sarebbe di fare il typecheck di ogni elemento di l,
              con una List.forall, e poi costruire il set Set(t, l). In questo caso facciamo 
              l'operazione equivalente, prima costruiamo il set, e poi facciamo il 
              typechecking*)
            let s = Set(t, l) in
            if typecheck(TSet(t), s) then s
            else raise ( RuntimeError "Wrong type")
    else raise ( RuntimeError "Duplicated elements")



(* Verifica se un insieme contiene un elemento dato *)
let set_contains(s, v) = 
    match s with
    |Set(t, l) -> 
        if typecheck(TSet(t), s) && typecheck(upcast t, v)
        then Bool(list_contains l v)
        else raise ( RuntimeError "Wrong type")
    |_ -> raise ( RuntimeError "Wrong type")

(* Inserisce un elemento in un insieme *)
let set_insert(s, v) = 
    match s with
    |Set(t, l) -> 
        if typecheck(TSet(t), s) && typecheck(upcast t, v)
        (*se v appartiene a s, restituisco s inalterato, altrimenti restituisco s U {v}*)
        then if list_contains l v 
                then Set(t, l)
                else Set(t, v::l)
        else raise ( RuntimeError "Wrong type")
    |_ -> raise ( RuntimeError "Wrong type")


(* Rimuove un elemento da un insieme *)
let set_remove(s, v) = 
    match s with
    |Set(t, l) -> 
        if typecheck(TSet(t), s) && typecheck(upcast t, v)
        then Set(t, list_remove l v)
        else raise ( RuntimeError "Wrong type")
    |_ -> raise ( RuntimeError "Wrong type")



(* Verifica se un insieme è vuoto *)
let is_empty (s : evT) = 
    (*andrebbe, come nelle altre primitive, fatto prima il typechecking
      ma la funzione è così banale che lo saltiamo. Don't do this at home.*)
    match s with
     Set(_, []) -> Bool(true) (*se è un set vuoto, restituisco true*)
    |Set(_, _) -> Bool(false) (*se è un set non vuoto, restituisco false*)
    |_ -> raise ( RuntimeError "Wrong type") (*se non è un set, errore*)

(* Verifica se un insieme è sottoinsieme di un altro insieme *)
let set_is_subset (s1, s2) = 
    match s1, s2 with
    | Set(t1, l1), Set(t2, l2) -> 
        if typecheck(TSet(t1), s1) && typecheck(TSet(t1), s2)
            then Bool(is_contained l1 l2)
            else raise ( RuntimeError "Wrong type")
    | _ -> raise ( RuntimeError "Wrong type")


(* Restituisce l'intersezione fra due sottoinsiemi*)
let set_intersection(s1, s2) = 
    let rec list_intersection(l1,l2) = match l1 with
        |[] -> []
        |h::tl -> if list_contains l2 h then h::list_intersection(tl,l2) else list_intersection(tl,l2)
    in match s1, s2 with
    |Set(t1, l1), Set(t2, l2) -> 
        if typecheck(TSet(t1), s1) && typecheck(TSet(t1), s2)
        then Set(t1, list_intersection(l1, l2))
        else raise ( RuntimeError "Wrong type")
    |_ -> raise ( RuntimeError "Wrong type")
    
(* Restituisce l'unione fra due sottoinsiemi*)
let set_union(s1, s2) = 
    let rec list_union(l1, l2) = match l1 with
        |[] -> l2
        |h::tl -> if list_contains l2 h then list_union(tl,l2) else h::list_union(tl, l2)
    in match s1, s2 with
    |Set(t1, l1), Set(t2, l2) -> 
        if typecheck(TSet(t1), s1) && typecheck(TSet(t1), s2)
        then Set(t1, list_union(l1, l2))
        else raise ( RuntimeError "Wrong type")
    |_ -> raise ( RuntimeError "Wrong type")

val is_zero : evT -> evT = <fun>


val int_eq : evT * evT -> evT = <fun>


val int_plus : evT * evT -> evT = <fun>


val int_sub : evT * evT -> evT = <fun>


val int_times : evT * evT -> evT = <fun>


...

val set_remove : evT * evT -> evT = <fun>


val is_empty : evT -> evT = <fun>


val set_is_subset : evT * evT -> evT = <fun>


val set_intersection : evT * evT -> evT = <fun>


val set_union : evT * evT -> evT = <fun>


### Es 6: Interprete

Completare la funzione eval, facendo pattern matching dei nuovi casi definiti dentro exp, e associandoli alle primitive definite precedentemente.

#### Soluzione:

La funzione `eval` è compie una semplice visita post-order dell'albero: per ogni nodo, valuta ricorsivamente i sottoalberi, e poi combina il risultato con una primitiva.

In [11]:
(* Interprete *)
let rec eval (e:exp) (s:evT env) : evT = 
    match e with
    | EInt(n) -> Int(n)
    | CstTrue -> Bool(true)
    | CstFalse -> Bool(false)
    | EString(s) -> String(s)
    | Den(i) -> (s i)

    | Prod(e1,e2) -> int_times((eval e1 s), (eval e2 s))
    | Sum(e1, e2) -> int_plus((eval e1 s), (eval e2 s))
    | Diff(e1, e2) -> int_sub((eval e1 s), (eval e2 s))
    | Div(e1, e2) -> int_div((eval e1 s), (eval e2 s))

    | IsZero(e1) -> is_zero (eval e1 s)
    | Eq(e1, e2) -> int_eq((eval e1 s), (eval e2 s))
    | LessThan(e1, e2) -> less_than((eval e1 s),(eval e2 s))
    | GreaterThan(e1, e2) -> greater_than((eval e1 s),(eval e2 s))

    | And(e1, e2) -> bool_and((eval e1 s),(eval e2 s))
    | Or(e1, e2) -> bool_or((eval e1 s),(eval e2 s))
    | Not(e1) -> bool_not(eval e1 s)

    | IfThenElse(e1,e2,e3) -> 
        let g = eval e1 s in 
            (match (typecheck(TBool,g),g) with
            |(true, Bool(true)) -> eval e2 s
            |(true, Bool(false)) -> eval e3 s
            |(_,_) -> raise ( RuntimeError "Wrong type")
            )
    | Let(i, e, ebody) -> eval ebody (bind s i (eval e s))
    | Fun(arg, ebody) -> Closure(arg,ebody,s)
    | Letrec(f, arg, fBody, leBody) ->
        let benv = bind (s) (f) (RecClosure(f, arg, fBody,s)) in
            eval leBody benv
    | Apply(eF, eArg) ->
        let fclosure = eval eF s in 
            (match fclosure with 
            | Closure(arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in 
                let aenv = bind fDecEnv arg aVal in 
                eval fbody aenv 
            | RecClosure(f, arg, fbody, fDecEnv) -> 
                let aVal = eval eArg s in
                let rEnv = bind fDecEnv f fclosure in
                let aenv = bind rEnv arg aVal in 
                eval fbody aenv
            | _ -> raise ( RuntimeError "Wrong type")
            )  
    | Empty(t) -> new_empty t
    | Singleton(t, e) -> 
        let v = eval e s in 
        new_singleton(t, v)
    | Of(t, e_list) ->
        let v_list = List.map (fun e -> eval e s) e_list
        in new_of(t, v_list)
    (*operatori*)
    |IsEmpty e -> 
        let v = eval e s in 
        is_empty v
    |Insert(e1, e2) ->
        let v1 = eval e1 s in
        let v2 = eval e2 s in 
        set_insert(v1, v2)
    |Remove(e1, e2) -> 
        let v1 = eval e1 s in
        let v2 = eval e2 s in 
        set_remove(v1, v2)
    |Member(e1, e2) -> 
        let v1 = eval e1 s in
        let v2 = eval e2 s in 
        set_contains(v1, v2)
    |Union(e1, e2) -> 
        let v1 = eval e1 s in
        let v2 = eval e2 s in 
        set_union(v1, v2)
    |Intersection(e1, e2) -> 
        let v1 = eval e1 s in
        let v2 = eval e2 s in 
        set_intersection(v1, v2)
    |IsSubset(e1, e2) -> 
        let v1 = eval e1 s in
        let v2 = eval e2 s in 
        set_is_subset(v1, v2)


val eval : exp -> evT env -> evT = <fun>


In [14]:
let e1 = Of(TString, [EString("Hello"); EString("World"); EString("from Minicaml"); ]);;
let e2 = Of(TString, [EString("Hello"); EString("World");]);;
let e3 = Let("s", Intersection(e1, e2), Member(Den("s"), EString("Hello")));;
(*let s = Intersection({string: "hello", "world", "from Minicaml"},{string: "hello", "world"}) in
  Member(s, "hello")*)

eval e3 emptyenv;;

val e1 : exp =
  Of (TString, [EString "Hello"; EString "World"; EString "from Minicaml"])


val e2 : exp = Of (TString, [EString "Hello"; EString "World"])


val e3 : exp =
  Let ("s",
   Intersection
    (Of (TString,
      [EString "Hello"; EString "World"; EString "from Minicaml"]),
    Of (TString, [EString "Hello"; EString "World"])),
   Member (Den "s", EString "Hello"))


- : evT = Bool true
